```python3
pip install -q sentencepiece
pip install -q transformers
```

# BERT 임베딩 생성하기

- 사전 학습된 BERT 모델을 다운로드 한다. 
- bert-base-uncased 모델은 12개의 인코더가 있는 모두 소문자로 변환한 uncased 토큰으로 변환된 BERT 기반 모델이다. 
- 표현 벡터 크기는 768이다.

In [1]:
import torch
from transformers import BertModel, BertTokenizer

model = BertModel.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


- bert-base-uncased 모델을 사전 학습 시키는데 사용된 토크나이저를 다운로드한다. 

In [2]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

**입력 전처리하기**

In [3]:
sentence = 'I love Korea'

문장을 토큰화하고 토큰을 얻는다. 

In [4]:
tokens = tokenizer.tokenize(sentence)
print(tokens)

['i', 'love', 'korea']


시작 부분에 [CLS] 토큰을 추가하고 토큰 목록 끝에 [SEP] 토큰을 추가한다. 

In [5]:
tokens = ['[CLS]'] + tokens + ['[SEP]']
print(tokens)

['[CLS]', 'i', 'love', 'korea', '[SEP]']


토큰 목록의 길이를 7로 유지해야 한다고 가정하면 끝이 2개의 [PAD] 토큰을 추가해야 한다. 

In [6]:
tokens = tokens + ['[PAD]'] + ['[PAD]']
print(tokens)

['[CLS]', 'i', 'love', 'korea', '[SEP]', '[PAD]', '[PAD]']


토큰이 [PAD] 토큰이 아니면 어텐션 마스크 값을 1로 설정하고, 그렇지 않으면 0으로 채운다. 

In [7]:
attention_mask = [1 if i!= '[PAD]' else 0 for i in tokens]
print(attention_mask)

[1, 1, 1, 1, 1, 0, 0]


모든 토큰을 다음과 같이 토큰 ID로 변환한다. 

In [8]:
token_ids = tokenizer.convert_tokens_to_ids(tokens)
print(token_ids)

[101, 1045, 2293, 4420, 102, 0, 0]


`token_ids` 와 `attention_mask` 를 텐서로 변환한다. 

In [9]:
token_ids = torch.tensor(token_ids).unsqueeze(0)
attention_mask = torch.tensor(attention_mask).unsqueeze(0)

print(token_ids, attention_mask)

tensor([[ 101, 1045, 2293, 4420,  102,    0,    0]]) tensor([[1, 1, 1, 1, 1, 0, 0]])


**임베딩 추출하기**

모델은 두 값으로 구성된 튜플로 출력을 반환한다. 첫 번째 값은 은닉 상태 표현인데, 이는 최종 인코더(12번째 인코더)에서 얻은 모든 토큰의 표현 벡터로 구성되어 있고, 두 번째 값은 [CLS] 토큰의 표현으로 구성된다. 

In [10]:
outputs = model(token_ids, attention_mask)

`hidden_rep` 는 입력에 대한 모든 토큰의 임베딩(표현)을 포함한다.  
`[1, 7, 768]`는 `[batch_size, sequence_length, hidden_size]` 를 의미한다. 

- `hidden_rep[0][0]` 은 첫 번째 토큰인 `[CLS]` 의 표현 벡터를 제공한다. 
- `hidden_rep[0][1]` 은 두 번째 토큰인 `I` 의 표현 벡터를 제공한다. 
- `hidden_rep[0][2]` 는 세 번째 토큰인 `love` 의 표현 벡터를 제공한다. 

In [11]:
hidden_rep = outputs.last_hidden_state
print(hidden_rep.shape)

torch.Size([1, 7, 768])


`cls_head` 에는 `[CLS]` 토큰의 표현이 포함된다.  
크기 `[1, 768]` 은 `[batch_size, hidden_size]` 를 나타낸다. 

`cls_head` 가 문장 전체의 표현을 보유하고 있다는 것을 배웠으므로 `cls_head` 를 `I love Korea` 문장의 표현 벡터로 사용할 수 있다. 

In [12]:
cls_head = outputs.pooler_output
print(cls_head.shape)

torch.Size([1, 768])


# BERT의 모든 인코더 레이어에서 임베딩을 추출하는 방법

모든 인코더 레이어에서 임베딩을 얻기 위해 사전 학습된 BERT 모델을 다운로드할 때 `output_hidden_states = True` 로 설정한다.

In [13]:
import torch
from transformers import BertModel, BertTokenizer

model = BertModel.from_pretrained('bert-base-uncased', output_hidden_states=True)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


앞에서 본 문장을 그대로 토큰화하고 시작 부분에 `[CLS]` 토큰을 추가하고 끝에 `[SEP]` 토큰을 추가한다.  
토큰의 길이를 7로 유지해야 한다고 가정하며 `[PAD]` 토큰을 추가한다. 

In [14]:
sentence = 'I love Korea'
tokens = tokenizer.tokenize(sentence)
tokens = ['[CLS]'] + tokens + ['[SEP]']
tokens = tokens + ['[PAD]'] + ['[PAD]']
print(tokens)

['[CLS]', 'i', 'love', 'korea', '[SEP]', '[PAD]', '[PAD]']


어텐션 마스크를 정의한다. 

In [15]:
attention_mask = [1 if i!= '[PAD]' else 0 for i in tokens]
print(attention_mask)

[1, 1, 1, 1, 1, 0, 0]


토큰을 토큰 ID로 변환한다. 

In [16]:
token_ids = tokenizer.convert_tokens_to_ids(tokens)
print(token_ids)

[101, 1045, 2293, 4420, 102, 0, 0]


`token_ids` 와 `attention_mask` 를 텐서로 변환한다. 

In [17]:
token_ids = torch.tensor(token_ids).unsqueeze(0)
attention_mask = torch.tensor(attention_mask).unsqueeze(0)

print(token_ids, attention_mask)

tensor([[ 101, 1045, 2293, 4420,  102,    0,    0]]) tensor([[1, 1, 1, 1, 1, 0, 0]])


모델은 다음과 같이 3개의 값이 있는 튜플을 반환한다. 
- 첫 번째 값인 `last_hidden_state` 는 최종 인코더 계층(12번째 인코더)에서만 얻은 모든 토큰의 표현을 가진다. 
- `pooler_output` 은 최종 인코더 계층의 `[CLS]` 토큰 표현을 나타내며 선형 및 tanh 활성화 함수에 의해 계산된다. 
- `hidden_states` 는 모든 인코더 계층에서 얻은 모든 토큰의 표현을 포함한다. 

In [18]:
outputs = model(token_ids, attention_mask)

last_hidden_state = outputs.last_hidden_state
pooler_output = outputs.pooler_output
hidden_states = outputs.hidden_states

`last_hidden_state` 는 최종 인코더 계층(12번째 인코더)에서만 얻은 모든 토큰의 표현을 가지고 있다.  
`[1, 7, 768]`는 `[batch_size, sequence_length, hidden_size]` 를 의미한다. 

- `hidden_rep[0][0]` 은 첫 번째 토큰인 `[CLS]` 의 표현 벡터를 제공한다. 
- `hidden_rep[0][1]` 은 두 번째 토큰인 `I` 의 표현 벡터를 제공한다. 
- `hidden_rep[0][2]` 는 세 번째 토큰인 `love` 의 표현 벡터를 제공한다. 

In [19]:
last_hidden_state.shape

torch.Size([1, 7, 768])

`pooler_output` 은 최종 인코더 계층의 `[CLS]` 토큰 표현을 포함하고 있으며 선형 및 tanh 활성화 함수에 의해 계산된다.  
`[1, 768]` 은 `[batch_size, hidden_size]` 를 나타낸다.

In [20]:
pooler_output.shape

torch.Size([1, 768])

마지막으로 모든 인코더 계층에서 얻은 모든 토큰의 표현을 포함하는 `hidden_states` 가 있다.  
이는 입력 임베딩 레이어($h_0$)에서 최종 인코더 레이어($h_12$)까지 모든 인코더 레이어의 표현을 포함하는 13개의 값을 포함하는 튜플이다. 

In [21]:
len(hidden_states)

13

- `hidden_states[0]` 는 입력 임베딩 레이어 $h_0$ 에서 얻은 모든 토큰의 표현 벡터를 가진다. 
- `hidden_states[1]` 는 첫 번째 인코더 계층 $h_1$ 에서 얻은 모든 토큰의 표현 벡터를 가진다. 
- `hidden_states[2]` 는 두 번째 인코더 계층 $h_2$ 에서 얻은 모든 토큰의 표현 벡터를 가진다. 
- `hidden_states[12]` 는 최종 인코더 레이어 $h_12$ 에서 얻은 모든 토큰의 표현 벡터를 가진다. 

`[1, 7, 768]`는 `[batch_size, sequence_length, hidden_size]` 를 의미한다. 

In [22]:
hidden_states[0].shape

torch.Size([1, 7, 768])